#  Notebook `06_predict.ipynb`

---

# 🎯 Prédiction avec les modèles finalisés

Ce notebook constitue une **passerelle entre la modélisation et l'application réelle**.  
Il vise à exploiter les modèles finalisés (régression + classification) pour **prédire le prix et la tranche de prix** d’un service Fiverr à partir de sa **description textuelle** et de son **indice de fiabilité**.

## 🎯 Objectifs

- 📦 Charger les **modèles enregistrés** (régression et classification)
- 🧠 Appliquer le **pipeline de transformation** cohérent avec l’entraînement
  - Embedding de la description (`sentence-transformers`)
  - Normalisation de la variable `Fiabilité`
  - Alignement des colonnes (`columns_used.pkl`)
- 🔮 Réaliser des **prédictions sur de nouvelles données**
  - Prédiction du **prix exact** (`Prix`) via un modèle de régression
  - Prédiction de la **tranche de prix** (`Tranche`) via un modèle de classification
- 🚀 Préparer les **fonctions d’appel** à utiliser dans l'application Gradio (`predict.py`)

## 🧠 Choix des modèles retenus

Les modèles utilisés ont été **sélectionnés après comparaison** dans les notebooks précédents :

- 📘 `03_model_regression.ipynb`  
  ➤ Modèle retenu : **Gradient Boosting Regressor**  
  ➤ Précis et robuste sur les indicateurs MAE / RMSE

- 📘 `04_model_classification.ipynb`  
  ➤ Modèle retenu : **Decision Tree Classifier**  
  ➤ Précision > 96 %, facile à interpréter

> 💡 Ces choix sont le fruit d’une **comparaison systématique des performances** sur des données testées et validées.

## ✅ Compétences mobilisées

- **Bloc 3 — C1** : Sélectionner l’algorithme le plus adapté en fonction de la problématique et des performances.
- **Bloc 3 — C2** : Appliquer un pipeline de transformation cohérent entre entraînement et prédiction.
- **Bloc 3 — C3** : Exploiter un modèle entraîné pour produire une prédiction adaptée au besoin métier.
- **Bloc 5 — C1** : Utiliser des embeddings pour transformer des descriptions textuelles en données numériques.

Ce notebook est **prérequis à l'intégration dans une interface applicative** (Gradio ou API Flask).

In [ ]:
import pandas as pd
import joblib
from sentence_transformers import SentenceTransformer

# 📦 Chargement des objets enregistrés
reg_model = joblib.load("models/regression/gradient_boosting.pkl")  # Modèle de régression
clf_model = joblib.load("models/classification/decision_tree.pkl")  # Modèle de classification
scaler = joblib.load("models/regression/scaler.pkl")                # Scaler pour Fiabilité
columns = joblib.load("models/columns_used.pkl")                    # Liste des colonnes attendues
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")  # Embedding

# 🔧 Fonction de prétraitement des données d'entrée
def preprocess_input(description: str, fiabilite: float) -> pd.DataFrame:
    # ➤ Génération de l'embedding de la description
    emb = embedding_model.encode([description])
    emb_dict = {f"emb_{i}": emb[0][i] for i in range(384)}
    
    # ➤ Construction du DataFrame d'entrée avec la fiabilité pondérée
    row = {**emb_dict, "Fiabilite": fiabilite}
    df = pd.DataFrame([row])

    # ➤ Réindexation selon les colonnes utilisées à l'entraînement (ajout de 0 si nécessaire)
    df = df.reindex(columns=columns, fill_value=0)

    # ➤ Mise à l’échelle de la colonne 'Fiabilite'
    df[["Fiabilite"]] = scaler.transform(df[["Fiabilite"]])
    return df

# 🔮 Fonction de prédiction du prix (régression)
def predict_price(description: str, fiabilite: float) -> float:
    fiabilite_pondérée = fiabilite * 0.8  # Pondération conservée (ex. : Beginner)
    X = preprocess_input(description, fiabilite_pondérée)
    return round(reg_model.predict(X)[0] * 10, 2)  # Multiplication pour retrouver l'échelle

# 🔮 Fonction de prédiction de la tranche (classification)
def predict_tranche(description: str, fiabilite: float) -> str:
    fiabilite_pondérée = fiabilite * 0.8  # Pondération identique à la régression
    X = preprocess_input(description, fiabilite_pondérée)
    return str(clf_model.predict(X)[0])  # Retourne : "Basse", "Moyenne" ou "Haute"